## dp实施原则

三个原则实施DP:
```
1. 可划分阶段,(理解就是可划分子问题,可递推)
2. 最优化原理,即子问题的局部最优将导致整个问题的全局最优，即问题具有最优子结构的性质,
   也就是说一个问题的最优解只取决于其子问题的最优解
3. 无后效性,即当前的状态只和过去有关,和未来无关,有点像线性时不变里的因果性
```

## 模型

上升,下降,或者其他场景都应该借鉴,举一反三

### 最长上升子序列的长度(不一定连续)


```
最长上升子序列可能以任何位置作为结尾! 这就是`划分子问题`(方式并不唯一!), 即: dp[n]是以a[n]结尾的最长子序列的长度;

dp[n]可以递推, 其值可能和dp[0],dp[1],...dp[n-1]有关, 这是`无后效性`

所有的dp[n]求得时,全局问题自然得解,即max(dp)
```

最重要的就是找出dp[i]的定义,和递推关系

dp[i]表示以a[i]结尾的上升子序列的长度, max(dp)即求解问题

```
step: 0 , dp[0] = 1
step: i, 对于j in range(1,i) a[j]是dp[j]代表的最长子序列的结尾, if a[i] > a[j],此时可以形成新的最长子序列 长度为a[j]+1,
    找到所有dp[j]中,可能的最大的dp[j]+1, 用来更新dp[i]
```

In [1]:
class Solution:
    def maxSubArrayLength(self,nums):
        dp = []
        [dp.append(1) for i in range(0,len(nums))]
        print(dp)
        for i in range(1,len(nums)):
            for j in range(1,i):
                dp[i] = max(dp[i],dp[j]+1) if nums[i] > nums[j] else dp[i]
        print(dp)
        return max(dp)

### 子序列的最大和


这个要更简单点,因为不考虑连续性:,
    dp[i] 表示以 a[i]结尾的子序列的和
```
step 0: dp[0] = 1
step i: dp[i] = dp[i-1] + num[i] if  dp[i-1] > 0 else num[i]
```
为何要dp[i-1] > 0 ? 如果dp[i]小于0,说明前面序列累加起来再加上现在的,比现在的还小,最长子序列不成立了.

In [ ]:
class Solution:
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        dp = nums.copy()
        for i in range(1, len(nums)):
            if dp[i-1] > 0:
                dp[i] = dp[i-1] + nums[i]
        return max(dp)

### 最长连续上升子序列的最大和

和上面的比,仅仅增加是否连续的判断

```
dp[i] = dp[i-1] + nums[i] if nums[i] < nums[i-1] and dp[i-1] > 0 else nums[i]
```

In [3]:
class Solution:
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        dp = []
        l = len(nums)
        dp = nums.copy()
        dp[0] = nums[0]
        for i in range(1,l):
            dp[i] = dp[i-1] + nums[i] if nums[i] < nums[i-1] and dp[i-1] > 0 else nums[i]
        return max(dp)

### 应用

### 合唱团问题

N个成员的合唱团,身高为a[i], 希望拿掉(N-K)个成员,剩余成员构成'合唱团',即
```
a1 < a2 < .. ai
ai > ai+1 >...aK
```
求K的最大值?

分析: 是2个子问题(上面2.1)的结合:
```
dp[i] 表示以a[i]为结尾的递增子序列(不用连续)的队列长度
dq[i] 表示以a[i]为开头的递减子序列(不用连续)的队列长度

求解问题为max(d[i]+dq[i]-1) 减1是因为i重复算了1次

```

## 参考

1. https://blog.csdn.net/cr496352127/article/details/77934132?locationNum=10&fps=1